<a href="https://colab.research.google.com/github/Arziva/Fundus_Ensemble/blob/main/Ensemble_Fundus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Activation, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/RETINA MODELS'

/content/drive/MyDrive/RETINA MODELS


MSAN Model

In [ ]:
attention_model = load_model('Copy of FINAL_MSAN.h5')

In [ ]:
colab_model_path = '/content/FINAL_MSAN.h5'
file_size = os.path.getsize(colab_model_path)
print(f"File size: {file_size} bytes")

TL models

In [ ]:
transfer_learning_model = load_model('Copy of FINALVGG16_93.00.h5')

In [ ]:
transfer_learning_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 256)               8388864   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Total params: 23104323 (88.14 MB)
Trainable params: 8389635 (32.00 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
attention_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_30 (Conv2D)          (None, 254, 254, 32)         896       ['input_7[0][0]']             
                                                                                                  
 max_pooling2d_24 (MaxPooli  (None, 127, 127, 32)         0         ['conv2d_30[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 conv2d_31 (Conv2D)          (None, 125, 125, 64)         18496     ['max_pooling2d_24[0][0]

In [ ]:
##########################Set the paths to your dataset
dataset_path = '/content/drive/MyDrive/Fundus_Sub'
dme_path = os.path.join(dataset_path, 'FUNDUS_DME')
dr_path = os.path.join(dataset_path, 'FUNDUS_DR')
healthy_path = os.path.join(dataset_path, 'FUNDUS_HEALTHY')

# Set the image dimensions
image_width, image_height = 100,100

###############################
images = []
labels = []

for class_path, class_label in [(dme_path, 0), (dr_path, 1), (healthy_path, 2)]:
    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (image_width, image_height))
        images.append(image)
        labels.append(class_label)

# Convert the data to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Perform train-test split
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
test_images_resized = [cv2.resize(image, (256, 256)) for image in test_images]


In [ ]:
from keras.layers import Input

# Create a new input layer with the desired shape
input_layer = Input(shape=(256, 256, 3))  # Modify the shape as needed

# Replace the existing input layer in your models with the new input layer
attention_model = Model(inputs=input_layer, outputs=attention_model.layers[-1].output)
transfer_learning_model = Model(inputs=input_layer, outputs=transfer_learning_model.layers[-1].output)


In [ ]:

attention_model_predictions = attention_model.predict(test_images)
transfer_learning_predictions = transfer_learning_model.predict(test_images)


ensemble_predictions = (attention_model_predictions + transfer_learning_predictions) / 2


ensemble_loss, ensemble_accuracy = ensemble_model.evaluate(test_images, test_labels, verbose=2)
print("Ensemble Test Loss:", ensemble_loss)
print("Ensemble Test Accuracy:", ensemble_accuracy)


In [ ]:
predicted_labels = np.argmax(ensemble_predictions, axis=1)
true_labels = np.argmax(test_labels, axis=1)


cm = confusion_matrix(true_labels, predicted_labels)


class_labels = ['DME', 'DR', 'Healthy']

#CM
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
#TT over epochs
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

report = classification_report(true_labels, predicted_labels, target_names=class_labels)
print(report)